In [2]:

import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [3]:

def get_coordinate(addr):
    browser = webdriver.Chrome(executable_path='chromedriver',options=options)
    browser.get("http://www.map.com.tw/")
    search = browser.find_element_by_id("searchWord")
    search.clear()
    search.send_keys(addr)
    browser.find_element_by_xpath("/html/body/form/div[10]/div[2]/img[2]").click() 
    time.sleep(2)
    iframe = browser.find_elements_by_tag_name("iframe")[1]
    browser.switch_to.frame(iframe)
    coor_btn = browser.find_element_by_xpath("/html/body/form/div[4]/table/tbody/tr[3]/td/table/tbody/tr/td[2]")
    coor_btn.click()
    coor = browser.find_element_by_xpath("/html/body/form/div[5]/table/tbody/tr[2]/td")
    coor = coor.text.strip().split(" ")
    lat = coor[-1].split("：")[-1]
    log = coor[0].split("：")[-1]
    browser.quit()
    return (lat, log)

In [6]:
get_coordinate("臺北市中正區延平南路96號")

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
import csv
# 開啟輸出的 CSV 檔案
with open('output2.csv', 'w', newline='') as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

    # 寫入一列資料
    writer.writerow(['緯緯度'])

    # 開啟 CSV檔案
    with open('監視器.csv', newline='',encoding="utf-8") as csvfile:

                # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile)
        for row in rows:
            print(get_coordinate(row[5]))
            writer.writerow(get_coordinate(row[5]))

('24.98972977', '121.57004896')
('24.98972977', '121.57004896')


In [ ]:
import requests

if __name__ == "__main__":
    res = requests.get(
        "http://api.map.com.tw/net/GraphicsXY.aspx?",
        data={
            'search_class': 'address',
            'searchkey': 'abc',
            'fun': 'funB',
            'SearchWord': '新北市中和區中正路716號'
        },
        headers={
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6',
            'Connection':'keep-alive',
            'Host':'api.map.com.tw',
            'Referer':'http://api.map.com.tw/API_Sample/Sample/GetAddressCoordinates.html',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
            }
    )
    print(res.text)

In [4]:
import requests
import re
import csv
import json
import time


def addrTo(addr):
    res = requests.get(
        "http://api.map.com.tw/net/GraphicsXY.aspx?",
        data={
            'search_class': 'address',
            'searchkey': 'abc',
            'fun': 'funB',
            'SearchWord': addr
        },
        headers={
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6',
            'Connection': 'close',
            'Host': 'api.map.com.tw',
            'Referer': 'http://api.map.com.tw/API_Sample/Sample/GetAddressCoordinates.html',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
        }
    )
    return res.text


if __name__ == "__main__":

    with open('output.csv', 'w', encoding='UTF-8', newline='') as write_file:
        writer = csv.writer(write_file)
        writer.writerow(['地址', 'Lat緯度', 'Lng經度'])
        count = 1

        with open('臺北市政府警察局錄影監視系統設置區位.csv', 'r', newline='') as csvfile:
            # 讀取 CSV 檔案內容
            rows = csv.reader(csvfile)
            for row in rows:
                # 捕捉到地址
                addr = '臺北市'+row[2]
                # print(row[3])

                # 透過捕捉到地址放入函式中取得json資料
                json_data = addrTo(addr)
                # print(json)
                # time.sleep(0)

                # 利用正則來擷取我們所需的經緯度資料
                # \d+\.?\d* 小數點形式的數字
                try:
                    m_Lat = re.search(r'(?<=lat":").*?(\d+\.?\d*)', json_data)
                    m_Lng = re.search(r'(?<=lng":").*?(\d+\.?\d*)', json_data)
                    print(addr)
                    print(m_Lat.group())
                    print(m_Lng.group())
                    print(count)
                    count += 1
                    writer.writerow([addr, m_Lat.group(), m_Lng.group()])
                except Exception as e:
                    print(json_data)
                    print(e)

ConnectionError: HTTPConnectionPool(host='api.map.com.tw', port=80): Max retries exceeded with url: /net/GraphicsXY.aspx (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00A48730>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))